In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
DIR_MAIN = '/content/drive/MyDrive/CCPS/SplitDataset'

TEST_DIR = DIR_MAIN + '/test'
TRAIN_DIR = DIR_MAIN + '/train'
VAL_DIR = DIR_MAIN + '/val'

# TEST_B = DIR_MAIN + '/test/Benign'
# TEST_E = DIR_MAIN + '/test/Early'
# TEST_Pre = DIR_MAIN + '/test/Pre'
# TEST_Pro = DIR_MAIN + '/test/Pro'


# TRAIN_B = DIR_MAIN + '/train/Benign'
# TRAIN_E = DIR_MAIN + '/train/Early'
# TRAIN_Pre = DIR_MAIN + '/train/Pre'
# TRAIN_Pro = DIR_MAIN + '/train/Pro'


# VAL_B = DIR_MAIN + '/val/Benign'
# VAL_E = DIR_MAIN + '/val/Early'
# VAL_Pre = DIR_MAIN + '/val/Pre'
# VAL_Pro = DIR_MAIN + '/val/Pro'

In [ ]:
# img_height,img_width=180,180
img_height,img_width=224,224
batch_size=32

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)

train_data_gen = image_gen_train.flow_from_directory(
    batch_size = batch_size,
    directory = TRAIN_DIR,
    shuffle= True,
    target_size = (img_height,img_width),
    class_mode = 'categorical')


image_generator_validation = ImageDataGenerator(rescale=1./255)

val_data_gen = image_generator_validation.flow_from_directory(
    batch_size=batch_size,
    directory=VAL_DIR,
    target_size=(img_height, img_width),
    class_mode='categorical')

image_gen_test = ImageDataGenerator(rescale=1./255)

test_data_gen = image_gen_test.flow_from_directory(
    batch_size=batch_size,
    directory=TEST_DIR,
    target_size=(img_height, img_width),
    class_mode='categorical')

Found 2263 images belonging to 4 classes.
Found 649 images belonging to 4 classes.
Found 330 images belonging to 4 classes.


In [ ]:
class_indices = test_data_gen.class_indices
class_names = list(class_indices.keys())
print(class_names)

['Benign', 'Early', 'Pre', 'Pro']


# ResNet50

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(240,240,3),
                   pooling='avg',classes=len(class_names),
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(len(class_names), activation='softmax'))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 24,638,852
Trainable params: 1,051,140
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = resnet_model.fit(
  train_data_gen,
  validation_data=val_data_gen,
  epochs=epochs
)

Epoch 1/10
71/71 [==============================] - 454s 6s/step - loss: 0.8831 - accuracy: 0.6774 - val_loss: 0.7058 - val_accuracy: 0.7550
Epoch 2/10
71/71 [==============================] - 444s 6s/step - loss: 0.6370 - accuracy: 0.7574 - val_loss: 0.5916 - val_accuracy: 0.7797
Epoch 3/10
71/71 [==============================] - 393s 6s/step - loss: 0.5724 - accuracy: 0.7848 - val_loss: 0.5551 - val_accuracy: 0.7766
Epoch 4/10
71/71 [==============================] - 454s 6s/step - loss: 0.5361 - accuracy: 0.7985 - val_loss: 0.5082 - val_accuracy: 0.8105
Epoch 5/10
71/71 [==============================] - 445s 6s/step - loss: 0.5095 - accuracy: 0.8051 - val_loss: 0.4931 - val_accuracy: 0.7966
Epoch 6/10
71/71 [==============================] - 437s 6s/step - loss: 0.4969 - accuracy: 0.8007 - val_loss: 0.5339 - val_accuracy: 0.7827
Epoch 7/10
71/71 [==============================] - 435s 6s/step - loss: 0.4700 - accuracy: 0.8206 - val_loss: 0.5559 - val_accuracy: 0.7643
Epoch 8/10
71

In [ ]:
ALL_SAVED_MODEL_PATH = '/content/drive/MyDrive/CCPS/SavedModels/ResNet50_224.h5'
resnet_model.save(ALL_SAVED_MODEL_PATH)

In [ ]:
val_image_batch, val_label_batch = next(iter(val_data_gen))
true_label_ids = np.argmax(val_label_batch, axis=-1)
print("Validation batch shape:", val_image_batch.shape)

Validation batch shape: (32, 224, 224, 3)


In [ ]:
results = resnet_model.evaluate(val_image_batch, val_label_batch)

1/1 [==============================] - 6s 6s/step - loss: 0.6219 - accuracy: 0.7188


In [ ]:
# Print the accuracy
accuracy = results[1]  # Assuming accuracy is the second metric
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 71.88%
